In [ ]:
import json
import pandas as pd
import requests

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

s = requests.session()

def get_bribe(url, s, tx_hash, miner_lower):
    data = {
        "method": "trace_replayTransaction",
        "params": [tx_hash, ["stateDiff"]],
        "id": 1,
        "jsonrpc": "2.0",
    }
    resp = s.post(url, json=data)
    res = json.loads(resp.text)
    diff = res["result"]["stateDiff"]
    if diff.get(miner_lower, None):
        if "*" in diff[miner_lower]["balance"]:
            return (
                int(diff[miner_lower]["balance"]["*"]["to"], 16)
                - int(diff[miner_lower]["balance"]["*"]["from"], 16)
            ) / 10**18
    return 0

LABELS = {
    '0xa69babef1ca67a37ffaf7a485dfff3382056e78c': 'CEX_SCP',
    '0x51c72848c68a965f66fa7a88855f9f7784502a7f': 'CEX_wintermute',
    '0x1bf621aa9cee3f6154881c25041bb39aed4ca7cc': 'CEX_a7cc',
    '0xe8c060f8052e07423f71d445277c61ac5138a2e5': 'CEX_a2e5',
    '0xbeb5fd030ffb0fbc95d68113c1c796eff65526d7': 'CEX_26d7',
    '0x6f1cdbbb4d53d226cf4b917bf768b94acbab6168': 'CEX_6168',
    '0x9def7cde171841a9f0724124ca0b01a622d749e4': 'CEX_49e4',
    '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad': 'UniversalRouter',
    '0x6b75d8af000000e20b7a7ddf000ba900b4009a80': 'MEV_jaredfromsubway',
    '0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec': '1inch',
    '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
    '0xe592427a0aece92de3edee1f18e0157c05861564': 'SwapRouter',
    '0x1111111254eeb25477b68fb85ed929f73a960582': 'AggregationRouterV5',
    '0xdef1c0ded9bec7f1a1670819833240f027b25eff': 'ZeroEx',
}

In [ ]:

swaps = pd.read_csv('/data/hackthon/swaps/uni_v3/2024-04-01.csv')

dfs=[]

for bot,label in LABELS.items():
    bot = bot.lower()
    if label[:4]!='CEX_':
        continue
    botswap=swaps[swaps['to_address']==bot]
    dfs.append(botswap)

df=pd.concat(dfs).reset_index(drop=True)
df['bribe'] = 0.0
print(df)

In [ ]:
minerDf = pd.read_csv('/home/hruan/PBS/bribe_2024-04-01_2024-04-02.csv')
miners ={}

for i in range(len(minerDf)):
    miners[minerDf.loc[i,'block']]=minerDf.loc[i,'builder']

print(len(miners))

In [ ]:

for i in range(len(df)):
    txHash = df.loc[i,'tx_hash']
    blockNum = df.loc[i,'block_num']
    if blockNum not in miners:
        print('block not found',blockNum)
        continue
    bribe = get_bribe('http://localhost:8547', s, txHash, miners[blockNum].lower())
    df.loc[i,'bribe']=bribe
    if i%100==0:
        print(i)


In [ ]:
print(df)
df.to_csv('/data/hackthon/swaps/uni_v3/2024-04-01_bribe.csv',index=False)